In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2


d:\Users\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
def choose(channelindex,x_train,length,y_train):
    """choose images of same channel from three rdgs and depth into a group of 4 images"""
    im0=[(normalize(x_train[0][i][0][channelindex]),normalize(x_train[0][i][1][channelindex]),\
        normalize(x_train[0][i][2][channelindex]),normalize(x_train[1][i][channelindex]),y_train[0][i]) for i in range(length)]
    random.shuffle(im0)
    return im0

def choose2(positionindex,x_train,length,y_train):
    """choose rdg images of one angle, three channels and depth image of three channels"""
    im0=[(normalize(x_train[0][i][0][positionindex]),normalize(x_train[0][i][1][positionindex]),\
        normalize(x_train[0][i][2][positionindex]),normalize(x_train[1][i][0]),normalize(x_train[1][i][1]),normalize(x_train[1][i][2]),\
            y_train[0][i]) for i in range(length)]
    random.shuffle(im0)
    return im0

def lazychoose(length):
    """load all the images from lazydata and normalize"""
    total=[]
    for i in range(length):
        data=[]
        im0=torch.tensor(cv2.normalize(cv2.imread("lazydata/train/X/"+str(i)+"/rgb/0.png"), None, alpha=0, beta=1,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
        
        im0=torch.permute(im0,(2,0,1))
    
        im1=torch.tensor(cv2.normalize(cv2.imread("lazydata/train/X/"+str(i)+"/rgb/1.png"), None, alpha=0, beta=1,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
        
        im1=torch.permute(im1,(2,0,1))
      
        im2=torch.tensor(cv2.normalize(cv2.imread("lazydata/train/X/"+str(i)+"/rgb/2.png"), None, alpha=0, beta=1,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
        im2=torch.permute(im2,(2,0,1))
    
        depth=torch.tensor(cv2.normalize(np.load("lazydata/train/X/"+str(i)+"/depth.npy"), None, alpha=0, beta=1,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
        

        y=np.load("lazydata/train/Y/"+str(i)+".npy")
        data.append(im0)
        data.append(im1)
        data.append(im2)
        data.append(depth)
        data.append(y)
        total.append(data)
 
    train_loader = torch.utils.data.DataLoader(total, batch_size=1, shuffle=True)
    return train_loader

def normalize(image):
    """normalize function for choose and choose2"""
    image=np.array(image)
    mean=np.mean(image)
    var=np.mean(np.square(image-mean))
    image=(image-mean)/np.sqrt(var)

    return torch.tensor(image)


In [43]:


chan1=lazychoose(3396)


In [44]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [45]:
class CNN(nn.Module):
    """CNN with 2 layers"""
    def __init__(self, input_size, conv_feature, fc_feature, output_size):
        super(CNN, self).__init__()

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=conv_feature, kernel_size=50),
            nn.ReLU(),
            nn.BatchNorm2d(72),
            nn.Conv2d(in_channels=conv_feature, out_channels=conv_feature, kernel_size=10),
            nn.ReLU(),
            nn.BatchNorm2d(72),
            nn.MaxPool2d(kernel_size=20,stride=10),
            nn.Conv2d(in_channels=conv_feature, out_channels=conv_feature, kernel_size=5),
            nn.ReLU(),
            nn.BatchNorm2d(72),
            nn.Conv2d(in_channels=conv_feature, out_channels=conv_feature, kernel_size=5),
            nn.ReLU(),
            nn.BatchNorm2d(72),
            
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=conv_feature, out_channels=conv_feature, kernel_size=5), 
            nn.ReLU(),
            
            )
        self.linear_layers=nn.Sequential(
            nn.Linear(conv_feature*4*4, fc_feature),
            nn.ReLU(),
           
            nn.Linear(fc_feature, output_size),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        logits = self.linear_layers(x)
        return logits



def train(epoch, model, optimizer, chan):
    
    for batch_ind,a in enumerate(chan):
        
        image=torch.stack((a[0][:,0],a[0][:,1],a[0][:,2],\
            a[1][:,0],a[1][:,1],a[1][:,2],a[2][:,0],a[2][:,1],\
                a[2][:,2],a[3][:,0],a[3][:,1],a[3][:,2]),1)


        target=torch.squeeze(a[4])

        image, target = image.to(device), target.to(device)       

        output = model(image)
        loss = F.mse_loss(output.float(),target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_ind % 52 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_ind * len(image), len(chan.dataset),
                100. * batch_ind/ len(chan), loss.item()))
            

In [6]:
conv_features = 72
fc_features = 50
output_size = 12
model_cnn = CNN(224*224,conv_features,fc_features,output_size) 

model_cnn.to(device)
optimizer = torch.optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.7)
torch.cuda.empty_cache()
for epoch in range(0, 2):
    train(epoch, model_cnn, optimizer,train_loader)
    train_loader = torch.utils.data.DataLoader(chan1, batch_size=32, shuffle=True)

torch.save(model_cnn.state_dict(), "model_cnn_2.pt")


Train Epoch: 0 [0/3396 (0%)]	Loss: 1.240531
Train Epoch: 0 [800/3396 (23%)]	Loss: 0.545399
Train Epoch: 0 [1600/3396 (47%)]	Loss: -3.185933
Train Epoch: 0 [2400/3396 (70%)]	Loss: 0.328922
Train Epoch: 0 [3200/3396 (93%)]	Loss: -0.867656
Train Epoch: 1 [0/3396 (0%)]	Loss: 1.087387
Train Epoch: 1 [800/3396 (23%)]	Loss: 1.034269
Train Epoch: 1 [1600/3396 (47%)]	Loss: 0.745285
Train Epoch: 1 [2400/3396 (70%)]	Loss: 1.173002
Train Epoch: 1 [3200/3396 (93%)]	Loss: 0.939948


In [47]:
'''load pretrained weight'''
res=torchvision.models.resnet50(pretrained=True)
print(res.conv1)
res.conv1=nn.Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
print(res.conv1)
num_ftrs = res.fc.in_features
print(num_ftrs)
del res.fc
res.add_module("fc", nn.Sequential(nn.Linear(num_ftrs, 800),nn.ReLU(),nn.Linear(800, 200),nn.ReLU(),nn.Linear(200, 12)))


d:\Users\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Users\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
2048


In [50]:
"""train the resnet model"""
mo=res
mo.to(device)
optimizer = torch.optim.Adam(mo.parameters(),lr=0.000005)
torch.cuda.empty_cache()
for epoch in range(0, 2):
    train(epoch, mo, optimizer,chan1)
torch.save(mo.state_dict(), "model_cnn_2.pt")

C:\Users\Ruokai Gu\AppData\Local\Temp\ipykernel_3300\177351430.py:58: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([1, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(output.float(),target.float())


Train Epoch: 0 [0/3396 (0%)]	Loss: 0.015337
Train Epoch: 0 [52/3396 (2%)]	Loss: 0.000904
Train Epoch: 0 [104/3396 (3%)]	Loss: 0.000325
Train Epoch: 0 [156/3396 (5%)]	Loss: 0.000369
Train Epoch: 0 [208/3396 (6%)]	Loss: 0.000372
Train Epoch: 0 [260/3396 (8%)]	Loss: 0.001400
Train Epoch: 0 [312/3396 (9%)]	Loss: 0.001334
Train Epoch: 0 [364/3396 (11%)]	Loss: 0.000414
Train Epoch: 0 [416/3396 (12%)]	Loss: 0.000296
Train Epoch: 0 [468/3396 (14%)]	Loss: 0.000432
Train Epoch: 0 [520/3396 (15%)]	Loss: 0.000322
Train Epoch: 0 [572/3396 (17%)]	Loss: 0.000200
Train Epoch: 0 [624/3396 (18%)]	Loss: 0.000363
Train Epoch: 0 [676/3396 (20%)]	Loss: 0.000120
Train Epoch: 0 [728/3396 (21%)]	Loss: 0.000235
Train Epoch: 0 [780/3396 (23%)]	Loss: 0.000629
Train Epoch: 0 [832/3396 (24%)]	Loss: 0.000502
Train Epoch: 0 [884/3396 (26%)]	Loss: 0.000495
Train Epoch: 0 [936/3396 (28%)]	Loss: 0.000252
Train Epoch: 0 [988/3396 (29%)]	Loss: 0.000417
Train Epoch: 0 [1040/3396 (31%)]	Loss: 0.000693
Train Epoch: 0 [1092/3

In [51]:
model=torchvision.models.resnet50(pretrained=False)

model.conv1=nn.Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
print(model.conv1)
num_ftrs = model.fc.in_features
del model.fc
model.add_module("fc", nn.Sequential(nn.Linear(num_ftrs, 800),nn.ReLU(),nn.Linear(800, 200),nn.ReLU(),nn.Linear(200, 12)))

model.to('cpu')
model1 = model
model1.load_state_dict(torch.load('model_cnn_2.pt'))
model1.eval()


d:\Users\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Users\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


ResNet(
  (conv1): Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [52]:
import pickle
def lazychoose_test(length):
    """load testdata and normalize the same way as train"""
    total=[]
    for i in range(length):
        data=[]
        im0=torch.tensor(cv2.normalize(cv2.imread("lazydata/test/X/"+str(i)+"/rgb/0.png"), None, alpha=0, beta=1,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
        
        im0=torch.permute(im0,(2,0,1))
 
        im1=torch.tensor(cv2.normalize(cv2.imread("lazydata/test/X/"+str(i)+"/rgb/1.png"), None, alpha=0, beta=1,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
        
        im1=torch.permute(im1,(2,0,1))

        im2=torch.tensor(cv2.normalize(cv2.imread("lazydata/test/X/"+str(i)+"/rgb/2.png"), None, alpha=0, beta=1,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
        im2=torch.permute(im2,(2,0,1))
    
        depth=torch.tensor(cv2.normalize(np.load("lazydata/test/X/"+str(i)+"/depth.npy"), None, alpha=0, beta=1,
                             norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F))
        index=pickle.load(open("lazydata/test/X/"+str(i)+"/field_id.pkl","rb"))
        data.append(im0)
        data.append(im1)
        data.append(im2)
        data.append(depth)
        data.append(index)
        
        total.append(data)
    train_loader = torch.utils.data.DataLoader(total, batch_size=64, shuffle=False)
    return train_loader

In [53]:
test_data=lazychoose_test(849)

In [54]:
preds = []

for data in test_data:
    image=torch.stack((data[0][:,0],data[0][:,1],data[0][:,2],\
            data[1][:,0],data[1][:,1],data[1][:,2],data[2][:,0],data[2][:,1],\
                data[2][:,2],data[3][:,0],data[3][:,1],data[3][:,2]),1)

    output = model1(image.to('cpu'))
  
    preds.append(output.cpu().detach().numpy())

In [55]:
file_ids=[]
for i in test_data:
    for j in i[4]:
    
        file_ids.append(j)
    

In [56]:

import pandas as pd
# model=model_cnn
outfile = 'submission.csv'

output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']


df = pd.concat([pd.DataFrame(file_ids), pd.DataFrame.from_records(np.concatenate(preds))], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

Written to csv file submission.csv
